In [1]:
import numpy as np
import torch
import os
import torch.nn as nn
import matplotlib.pyplot as plt
import random
from torch.optim import LBFGS
from tqdm import tqdm
from torchviz import make_dot

script_dir = os.getcwd()

from pinnsform.util import *
from pinnsform.model import PINN_base

In [2]:
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

device = 'cuda'

In [3]:
mesh, boundaries = generate_mesh_object((101,101), domain=([0, 2*np.pi], [0, 1]), device=device, full_requires_grad=True, border_requires_grad=False)

b_left = boundaries[0][0]
b_right = boundaries[0][1]
initial = boundaries[1][0]

/upb/users/p/pauldf/profiles/unix/cs/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
RHO = 5.0

def loss_fn(model, mesh, b_left, b_right, initial, initial_values):
    # pde
    u = f(model, mesh)
    pde_residue = df(model, mesh, wrt=1) - RHO*u*(1.0-u)
    pde_loss = pde_residue.pow(2).mean()

    # boundary
    boundary_residue = f(model, b_left) - f(model, b_right)
    boundary_loss = boundary_residue.pow(2).mean()

    # initial
    initial_residue = f(model, initial) - initial_values
    initial_loss = initial_residue.pow(2).mean()

    return pde_loss, boundary_loss, initial_loss

def intial_value_function(x):
    return torch.exp(- (x - torch.pi)**2 / (2*(torch.pi/4.0)**2))

with torch.no_grad():
    initial_values = intial_value_function(initial.part[0])   #torch.exp(- (x_left[:,0] - torch.pi)**2 / (2*(torch.pi/4)**2))

loss_function = partial(loss_fn, mesh=mesh, b_left=b_left, b_right=b_right, initial=initial, initial_values=initial_values)

In [5]:
#res, b_left, b_right, b_upper, b_lower = get_data([0,2*np.pi], [0,1], 101, 101)
#res_test, _, _, _, _ = get_data([0,2*np.pi], [0,1], 101, 101)

#res = torch.tensor(res, dtype=torch.float32, requires_grad=True).to(device)
#b_left = torch.tensor(b_left, dtype=torch.float32, requires_grad=True).to(device)
#b_right = torch.tensor(b_right, dtype=torch.float32, requires_grad=True).to(device)
#b_upper = torch.tensor(b_upper, dtype=torch.float32, requires_grad=True).to(device)
#b_lower = torch.tensor(b_lower, dtype=torch.float32, requires_grad=True).to(device)

#x_res, t_res = res[:,0:1], res[:,1:2]
#x_left, t_left = b_left[:,0:1], b_left[:,1:2]
#x_right, t_right = b_right[:,0:1], b_right[:,1:2]
#x_upper, t_upper = b_upper[:,0:1], b_upper[:,1:2]
#x_lower, t_lower = b_lower[:,0:1], b_lower[:,1:2]

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [6]:
# Train PINNs 

model = PINN_base(in_dim=2, hidden_dim=512, out_dim=1, num_layer=4).to(device)

model.apply(init_weights)
optim = LBFGS(model.parameters(), line_search_fn='strong_wolfe')

print(model)
print(get_n_params(model))

PINN_base(
  (linear): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Tanh()
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
)
527361


In [7]:
loss_track = []

for i in tqdm(range(500)):
    def closure():
        loss_pde, loss_bc, loss_ic = loss_function(model)
        
        loss_track.append([loss_pde.item(), loss_bc.item(), loss_ic.item()])

        loss = loss_pde + loss_bc + loss_ic

        #graph = make_dot(loss)
        #graph.save(os.path.join(script_dir, f"graph_pinn_new_epoch_{i}.dot"))

        optim.zero_grad()
        loss.backward()
        return loss

    optim.step(closure)

100%|██████████| 500/500 [07:55<00:00,  1.05it/s] 


In [8]:
print('Loss Res: {:4f}, Loss_BC: {:4f}, Loss_IC: {:4f}'.format(loss_track[-1][0], loss_track[-1][1], loss_track[-1][2]))
print('Train Loss: {:4f}'.format(np.sum(loss_track[-1])))

torch.save(model.state_dict(), './1dreaction_pinns.pt')

Loss Res: 0.019411, Loss_BC: 0.000006, Loss_IC: 0.179779
Train Loss: 0.199195


In [ ]:
# Visualize PINNs 
res_test = torch.tensor(res_test, dtype=torch.float32, requires_grad=True).to(device)
x_test, t_test = res_test[:,0:1], res_test[:,1:2]

with torch.no_grad():
    pred = model(x_test, t_test)[:,0:1]
    pred = pred.cpu().detach().numpy()

pred = pred.reshape(101,101)

def h(x):
    return np.exp( - (x-np.pi)**2 / (2 * (np.pi/4)**2))

def u_ana(x,t):
    return h(x) * np.exp(5*t) / ( h(x) * np.exp(5*t) + 1 - h(x))

res_test, _, _, _, _ = get_data([0,2*np.pi], [0,1], 101, 101)
u = u_ana(res_test[:,0], res_test[:,1]).reshape(101,101)

rl1 = np.sum(np.abs(u-pred)) / np.sum(np.abs(u))
rl2 = np.sqrt(np.sum((u-pred)**2) / np.sum(u**2))

print('relative L1 error: {:4f}'.format(rl1))
print('relative L2 error: {:4f}'.format(rl2))

plt.figure(figsize=(4,3))
plt.imshow(pred, extent=[0,np.pi*2,1,0], aspect='auto')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Predicted u(x,t)')
plt.colorbar()
plt.tight_layout()
plt.savefig('./1dreaction_pinns_pred.png')
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow(u, extent=[0,np.pi*2,1,0], aspect='auto')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Exact u(x,t)')
plt.colorbar()
plt.tight_layout()
plt.savefig('./1dreaction_exact.png')
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow(np.abs(pred - u), extent=[0,np.pi*2,1,0], aspect='auto')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Absolute Error')
plt.colorbar()
plt.tight_layout()
plt.savefig('./1dreaction_pinns_error.png')
plt.show()